# Projet 2 : Préparez des données pour un organisme de santé publique

## Contexte

L'agence Santé publique France souhaite améliorer sa base de données Open Food Facts, qui est open source afin de permettre à tous de connaitre la qualité nutritionnelle des produits.

Aujourd'hui, pour ajouter un produit à la base de données d'Open Food Facts, il est nécessaire de remplir de nombreux champs textuels et numériques, ce qui peut conduire à des erreurs de saisie et à des valeurs manquantes.

L'agence Santé publique France confie à notre entreprise la création d'un système de suggestion ou d'auto-complétion pour aider les usagers à remplir plus efficacement la base de données, même si cette dernière venait à être légèrement modifiée.


Notre objectif : nettoyer et explorer les données en interne afin de déterminer la faisabilité de cette idée d'application.s. 

## Librairies utilisées avec le langage Python:

In [1]:
# Importation des librairies
import numpy as np
import pandas as pd
import pyarrow as pya
import matplotlib.pyplot as plt
import seaborn as sns

## Visualisation du tableau de données brutes


In [2]:
# Chargement des données brutes du fichier CSV avec des virgules pour séparateur
data=pd.read_csv('fr.openfoodfacts.org.products.csv', delimiter = '\t')
# Visualisation des 5 premières lignes
data.head(5)

C:\Users\celine.lesur\AppData\Local\Temp\ipykernel_22984\4034584380.py:2: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('fr.openfoodfacts.org.products.csv', delimiter = '\t')


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,4559,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,16087,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,16094,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Les champs sont séparés en quatre sections :

##### Les informations générales sur la fiche du produit :
- code : codebarre du produit ( EAN-13 ou  codes internes pour certains magasins). Pour les produits sans codebarres, Open Food Fact attribue un numéro commençant par le préfixe réservé 200.
- url : url de la page du produit sur Open Food Facts
- creator : contributeur qui a ajouté le produit pour la première fois
- created_t : date à laquelle le produit a été ajouté (timestamp)
- created_datetime : date à laquelle le produit a été ajouté (iso8601 format: yyyy-mm-ddThh:mn:ssZ)
- last_modified_t : date à laquelle le produit a été modifié pour la dernière fois (timestamp)
- last_modified_datetime : date à laquelle le produit a été modifié pour la dernière fois (iso8601 format: yyyy-mm-ddThh:mn:ssZ)
- product_name : nom du produit
- generic_name : nom générique
- quantity : quantité et unité

##### Un ensemble de tags (catégorie du produit, origine des ingrédients, liste des pays où il est vendu, ...)

##### Les ingrédients composants les produits et leurs additifs éventuels

##### Des informations nutritionnelles (quantité en gramme d'un nutriment pour 100 grammes du produit)

Pour plus d'informations sur les différentes variables : https://world.openfoodfacts.org/data/data-fields.txt

## Etude des données : nombre de lignes & colonnes, types des données et statistiques

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Columns: 162 entries, code to water-hardness_100g
dtypes: float64(106), object(56)
memory usage: 396.5+ MB


Nous avons donc un data frame composé de 162 variables dont 106 sont des type flottant, et 56 de type objet.
Dans ce data frame, 320771 produits ont été entrés.

Maintenant,recherchons d'éventuels duplicatas en fonction des codebarres produits.

In [4]:
data.duplicated('code').sum()

133

Il existe 133 duplicatas. Etudions les de plus près :

In [5]:
data_dup=data.duplicated('code')
# print(data_dup['data_dup'])
data_dup=np.array(data_dup)
print(type(data_dup))
x= np.where(data_dup == True)[0] # format array
print(x)

data.iloc[x].sort_values(by = 'code').head(133)

<class 'numpy.ndarray'>
[   481    519    560    632   1337   1343   2877   4547   6585   9892
  13067  13384  13391  13392  13394  13395  13396  19029  19063  19064
  20399  21580  21583  22824  27052  34613  34644  51325  62640  64799
  66964  68280  68290  71863  71872  71874  77489  79567  79568  79578
  79579  80834  83154  83165  87709  87711  87732  91156  91723  99206
  99408 101229 101254 120240 120390 120710 122219 126057 128464 128470
 134825 137332 137333 137334 137340 138106 138211 138216 158335 161112
 161115 168878 169195 174391 174392 174596 174719 174721 174990 174991
 175024 175025 175026 175149 175189 175235 176993 177550 177552 189103
 189109 189119 189152 189160 189162 189168 189242 189244 189248 189250
 189260 189262 189269 189272 189345 189362 189364 189379 189404 189406
 189417 229431 237297 237298 237299 251107 262433 263787 263798 263814
 263820 263825 264016 268065 275200 275201 275202 276819 280571 280572
 281938 300376 301337]


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
9892,11778,http://world-fr.openfoodfacts.org/produit/0011...,beniben,1474489149,2016-09-21T20:19:09Z,1491142385,2017-04-02T14:13:05Z,All Butter Reduced Fat Stem Ginger Cookies,NaN,225 g,...,NaN,NaN,NaN,NaN,NaN,NaN,18.0,18.0,NaN,NaN
481,16117,http://world-fr.openfoodfacts.org/produit/0001...,usda-ndb-import,1489065258,2017-03-09T13:14:18Z,1489065258,2017-03-09T13:14:18Z,Colossal Olives With Jalapeno Peppers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
519,24600,http://world-fr.openfoodfacts.org/produit/0002...,tacinte,1435406581,2015-06-27T12:03:01Z,1435406592,2015-06-27T12:03:12Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
560,31233,http://world-fr.openfoodfacts.org/produit/0003...,openfoodfacts-contributors,1451914937,2016-01-04T13:42:17Z,1491146137,2017-04-02T15:15:37Z,Super chicken spinach & quinoa,NaN,600 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
632,58001,http://world-fr.openfoodfacts.org/produit/0005...,kiliweb,1487432837,2017-02-18T15:47:17Z,1487432838,2017-02-18T15:47:18Z,Bramley Apple Crumble,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189364,NaN,NaN,villecomtal-sur-arros-gers-france,"Courrières,France",Cora,France,en:france,France,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189379,NaN,NaN,villecomtal-sur-arros-gers-france,NaN,NaN,France,en:france,France,"_Lait_ entier (78,1%), sucre (8,5%), _lait_ éc...","Lait, lait, lait, lait, lait",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189404,NaN,NaN,villecomtal-sur-arros-gers-france,"Brétigny-sur-Orge,Marseille 5°,France","Auchan,Super U","Suisse,France","en:france,en:switzerland","France,Suisse","_lait_ entier, sucre (9,6% en moyenne), fruits...","lait, lait, lait, lait",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189406,NaN,NaN,villecomtal-sur-arros-gers-france,"Marseille 5°,France",Super U,France,en:france,France,"_lait_ entier, sucre (9,7% en moyenne), fruits...","lait, lait, lait, lait",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.loc[data['code']==16117]

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
6,16117,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055730,2017-03-09T10:35:30Z,1489055730,2017-03-09T10:35:30Z,Organic Long Grain White Rice,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,16117,http://world-fr.openfoodfacts.org/produit/0001...,usda-ndb-import,1489065258,2017-03-09T13:14:18Z,1489065258,2017-03-09T13:14:18Z,Colossal Olives With Jalapeno Peppers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Un codebarre dupliqué ne correspond pas au même produit(product_name). Nous ne pouvons donc pas considérer ces duplicatas comme des doublons. Au vu de la petite quantité, nous les gardons dans la base de données pour l'analyse.

Tentons d'alleger la base de données, en ne gardant que les colonnes pertinentes pour l'analyse :
- dans la fiche produit, seul le nom du produit et sa quantité peuvent être intéressante

Regardonc si le nom doit être récupéré dans la colonne product_name ou generic_name :

In [7]:
variables = ['product_name', 'generic_name']
data[variables].isna().mean()

product_name    0.055373
generic_name    0.835413
dtype: float64

Il existe plus de valeurs manquantes dans la colonne generic_name. Il est donc plus pertinent de garder la colonne product_name pour notre analyse.

Cependant, nous pouvons également récupérer les valeurs de la colonne generic_name  et l'attribuer à la colonne product_name si cette dernière n'a pas de valeurs. Regardonc à combien de produits ce cas s'applique :

In [8]:
df=data[variables].isna()
df_missing_name=df.loc[(df['product_name']==True) & (df['generic_name']==False)]
df_missing_name.shape

(119, 2)

Nous avons donc 119 produits pour lequel le nom a été renseigné dans la colonne generic_name et sans valeur dans la colonne product_name.

Assignons ces valeurs dans la colonne product_name pour avoir une colonne la plus renseignée possible :

In [9]:
df_missing_name_index=df_missing_name.index.to_list()

In [14]:
# Fonction permettant de remplacer les valeurs manquantes d'une colonne par une deuxième et de supprimer la deuxième colonne
def rempl_missing_value(df,col1,col2) :
    variables = [col1, col2]
    df2=df[variables].isna()
    df_missing_name=df2.loc[(df2['product_name']==True) & (df2['generic_name']==False)]
    df_missing_name_index=df_missing_name.index.to_list()
    for i in df_missing_name_index :
        df.loc[i,col1] = df.loc[i,col2]    
    return df.drop([col2], axis=1)
            

In [15]:
# On appelle la fonction pour créer un nouveau dataframe nettoyé 
data_clean = rempl_missing_value(data,'product_name','generic_name')


(320772, 161)


Intéressons nous maintenant aux valeurs manquantes :

In [12]:
data_vm=data.isna().mean()
data_vm[data_vm>0.5].sort_values()

additives_fr            0.517788
additives_tags          0.517788
cholesterol_100g        0.550802
trans-fat_100g          0.553271
calcium_100g            0.560280
                          ...   
elaidic-acid_100g       1.000000
mead-acid_100g          1.000000
erucic-acid_100g        1.000000
lignoceric-acid_100g    1.000000
water-hardness_100g     1.000000
Length: 128, dtype: float64

En ce qui concerne les informations nutritionnelles, il parait assez logique de mettre à 0 les valeurs manquantes en considérant que si cela n'a pas été rempli c'est que la valeur est nulle.

In [13]:
# Imputation des valeur  Nan à 0 pour les informations nutritionnelles
data.iloc[:,63]

0            NaN
1         2243.0
2         1941.0
3         2540.0
4         1552.0
           ...  
320767       NaN
320768       0.0
320769       NaN
320770       NaN
320771    2092.0
Name: energy_100g, Length: 320772, dtype: float64


Liste des étapes:
- suggérer les valeurs manquantes pour une variable dont plus de 50% des valeurs sont manquantes.
- Repérer des variables pertinentes pour les traitements à venir, et nécessaires pour suggérer des valeurs manquantes,.
- Nettoyer les données en mettant en évidence les éventuelles valeurs manquantes parmi les variables pertinentes sélectionnées, avec au moins 3 méthodes de traitement adaptées aux variables concernées,
(imputation par 0, moyenne, médiane OU estimation avec ML de type KNN OU estimation via iterative imputer (pertinente que sur des variables corrélées entre elles)
- en identifiant et en traitant les éventuelles valeurs aberrantes de chaque variable.
- Automatiser ces traitements pour éviter de répéter ces opérations

- Tout au long de l’analyse, produire des visualisations afin de mieux comprendre les données et de les expliquer  à un public néophyte (attention à la lisibilité : taille des textes, choix des couleurs, netteté suffisante, et variez les graphiques (boxplots, histogrammes, diagrammes circulaires, nuages de points…)).
- Effectuer une analyse univariée pour chaque variable intéressante, afin de synthétiser son comportement + graphiques variables catégorielles et numériques
- Effectuer une analyse bivariée avec matrice de corrélations + graphiques croisant 2 variables numériques OU une numérique et une catégorielle
- Sélectionner / créer des variables à l’aide d’une analyse multivariée. Analyse descriptive en Composante Principale (ACP) / Analyse explicative ANOVA
Effectuer les tests statistiques appropriés pour vérifier la significativité des résultats + graphiques (ex : repartition du nutriscore, nb d'articles vendus dans chaque pays,...)

- Rédiger un rapport d’exploration et une conclusion pour expliquer la faisabilité de l’application demandée.

- Même si les données n’incluent pas de données personnelles, on doit expliquer dans une présentation en quoi ce projet respecte les 5 grands principes du RGPD. Santé publique France aimerait publier quelque chose sur le site Open Food Facts pour couper court aux questions sur le respect des RGPD que nous recevons parfois. 